# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [12]:
# Install gensim
!pip install -U gensim
%config Completer.use_jedi = False

Requirement already up-to-date: gensim in /opt/conda/lib/python3.8/site-packages (3.8.3)


In [13]:
# Load pretrained word vectors using gensim
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [14]:
# Explore the word vector for "king"
wiki_embeddings['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [15]:
# Find the words most similar to king based on the trained word vectors
wiki_embeddings.most_similar('king')

[('prince', 0.7682329416275024),
 ('queen', 0.7507690787315369),
 ('son', 0.7020887732505798),
 ('brother', 0.6985775232315063),
 ('monarch', 0.6977890729904175),
 ('throne', 0.691999077796936),
 ('kingdom', 0.6811410188674927),
 ('father', 0.680202841758728),
 ('emperor', 0.6712858080863953),
 ('ii', 0.6676074266433716)]

In [16]:
wiki_embeddings.most_similar('pansy')

[('lorelei', 0.5161976218223572),
 ('neang', 0.4673246145248413),
 ('fanta', 0.46462246775627136),
 ('itsy', 0.46324676275253296),
 ('damsel', 0.4551336169242859),
 ('mako', 0.45218175649642944),
 ('phuong', 0.45166337490081787),
 ('aqua', 0.4488627314567566),
 ('s.k.', 0.44639748334884644),
 ('bubblegum', 0.4431919455528259)]

### Train Our Own Model

In [17]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [18]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply(lambda x:gensim.utils.simple_preprocess(x))
messages.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g...","[go, until, jurong, point, crazy, available, only, in, bugis, great, world, la, buffet, cine, th..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives around here though","[nah, don, think, he, goes, to, usf, he, lives, around, here, though]"


In [19]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [20]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                  size=100,
                                  window=5,
                                  min_count=2)

In [21]:
# Explore the word vector for "king" base on our trained model
w2v_model.wv['king']

array([ 0.0213431 ,  0.02793939, -0.00294615, -0.03720459, -0.0002877 ,
        0.02675066,  0.00579894, -0.02713411, -0.00845478,  0.04408501,
       -0.01371462, -0.07255834, -0.04268394,  0.02626654,  0.09524437,
       -0.01582514, -0.03879341, -0.03665091, -0.03316345, -0.01134343,
        0.02175622,  0.03299017,  0.05437201,  0.00278176, -0.03493413,
        0.04382637, -0.03363669,  0.02955053, -0.0696837 , -0.06221814,
       -0.00128294, -0.03036061, -0.02864967,  0.00134192, -0.02836266,
       -0.02675285, -0.00596533,  0.02196117, -0.04117718,  0.03573877,
        0.08122399,  0.00030747, -0.00191515,  0.03503584,  0.06853321,
       -0.02706556,  0.00824319, -0.01293469,  0.08163768, -0.04566967,
        0.03407932,  0.02434064, -0.05162141, -0.03401947, -0.05039034,
       -0.00258773, -0.07787459, -0.02174657, -0.03611391, -0.03938799,
        0.04144573,  0.0006871 , -0.0090277 , -0.08294542, -0.030517  ,
        0.04100731,  0.10982145,  0.05064801, -0.00805382,  0.05

In [22]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('king')

[('gonna', 0.9981613159179688),
 ('top', 0.9980642199516296),
 ('points', 0.9980641603469849),
 ('ill', 0.9980623126029968),
 ('special', 0.998059093952179),
 ('girl', 0.9980579614639282),
 ('am', 0.9980196356773376),
 ('enjoy', 0.9980168342590332),
 ('wk', 0.9980154037475586),
 ('something', 0.9980146288871765)]